In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import matplotlib.pyplot as plt

# --- IMPORTS FROM YOUR NEW FILES ---
from stage1_dataset import SARToLDataset
from stage1_models import GeneratorResNet
from stage1_utils import SSIMLoss, evaluate_metrics, SSIMLoss

# --- Configuration ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATASET_PATH = r".\dataset"
IMAGE_SIZE = 256
BATCH_SIZE = 16 
NUM_WORKERS = 3
NUM_EPOCHS = 100
LEARNING_RATE = 2e-4
CHECKPOINT_DIR = './checkpoint_structure'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True

def main():
    # 1. Dataset & Dataloader
    full_ds = SARToLDataset(DATASET_PATH, IMAGE_SIZE)
    train_size = int(0.9 * len(full_ds))
    val_size = len(full_ds) - train_size
    train_ds, val_ds = random_split(full_ds, [train_size, val_size])
    
    # NOW SAFE to use num_workers=4 because Dataset is imported
    train_loader = DataLoader(
        train_ds, 
        batch_size=BATCH_SIZE, 
        shuffle=True, 
        num_workers=NUM_WORKERS, 
        pin_memory=True, 
        persistent_workers=True
    )
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

    # 2. Setup Model
    generator = GeneratorResNet().to(DEVICE)
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
    criterion_pixel = nn.L1Loss()
    scaler = torch.amp.GradScaler('cuda')

    # 3. Resume Logic
    start_epoch = 0
    best_ssim = 0.0
    resume_file = os.path.join(CHECKPOINT_DIR, "last.pth")
    criterion_pixel = nn.L1Loss()
    criterion_ssim = SSIMLoss()
    if os.path.exists(resume_file):
        print(f"--> Resuming training from {resume_file}")
        checkpoint = torch.load(resume_file, map_location=DEVICE, weights_only=False)
        
        # Handle dictionary loading safely
        if isinstance(checkpoint, dict) and 'generator_state_dict' in checkpoint:
            generator.load_state_dict(checkpoint['generator_state_dict'])
            optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
            if 'scaler_state_dict' in checkpoint:
                scaler.load_state_dict(checkpoint['scaler_state_dict'])
            start_epoch = checkpoint['epoch'] + 1
            best_ssim = checkpoint.get('best_ssim', 0.0)
            print(f"    Resuming at Epoch {start_epoch+1}. Best SSIM: {best_ssim:.4f}")
        else:
            print("    Old checkpoint format detected. Starting fresh to avoid errors.")
    else:
        print("--> No checkpoint found. Starting fresh.")

    print(f"Starting Structure Training on {DEVICE}...")

    # 4. Training Loop
    for epoch in range(start_epoch, NUM_EPOCHS):
        
        # --- NEW: Manual Learning Rate Scheduler ---
        if epoch < 20:
            current_lr = 2e-4
        elif epoch < 70:
            current_lr = 1e-4
        else:
            current_lr = 1e-5
            
        # Apply the learning rate to the optimizer
        for param_group in optimizer_G.param_groups:
            param_group['lr'] = current_lr
            
        generator.train()
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [LR={current_lr}]")
        
        for sar, target_L in loop:
            sar, target_L = sar.to(DEVICE, non_blocking=True), target_L.to(DEVICE, non_blocking=True)
            
            optimizer_G.zero_grad()
            
            with torch.amp.autocast('cuda'):
                fake_L = generator(sar)
                loss_l1 = criterion_pixel(fake_L, target_L)
                loss_ssim = criterion_ssim(fake_L, target_L)
                
                # --- NEW: SSIM Logic (Enable after 15 epochs) ---
                if epoch < 15:
                    # First 15 epochs: Pure pixel loss to get colors/shapes right
                    loss = loss_l1
                else:
                    # Afterward: Add structural consistency
                    loss = (1.0 * loss_l1) + (0.1 * loss_ssim)
            
            
            scaler.scale(loss).backward()
            scaler.step(optimizer_G)
            scaler.update()
            
            loop.set_postfix(loss=loss.item(), lr=current_lr)

        # 5. Validation
        generator.eval()
        val_psnr, val_ssim, val_enl = 0.0, 0.0, 0.0
        total_samples = 0
        
        with torch.no_grad():
            for sar, target in val_loader:
                sar = sar.to(DEVICE)
                fake_L = generator(sar)
                p, s, e, b = evaluate_metrics(fake_L, target)
                val_psnr += p; val_ssim += s; val_enl += e; total_samples += b

        avg_psnr = val_psnr / total_samples
        avg_ssim = val_ssim / total_samples
        
        print(f"--> Val Metrics: PSNR: {avg_psnr:.2f} | SSIM: {avg_ssim:.4f}")

        # Save Checkpoints
        checkpoint_data = {
            'epoch': epoch,
            'generator_state_dict': generator.state_dict(),
            'optimizer_G_state_dict': optimizer_G.state_dict(),
            'scaler_state_dict': scaler.state_dict(),
            'best_ssim': best_ssim
        }
        torch.save(checkpoint_data, os.path.join(CHECKPOINT_DIR, "last.pth"))
        # --- NEW: Save Every Epoch ---
        torch.save(checkpoint_data, os.path.join(CHECKPOINT_DIR, f"model_epoch_{epoch+1}.pth"))

        if avg_ssim > best_ssim:
            print(f"    New Best SSIM! ({avg_ssim:.4f}). Saving best.pth...")
            best_ssim = avg_ssim
            checkpoint_data['best_ssim'] = best_ssim
            torch.save(checkpoint_data, os.path.join(CHECKPOINT_DIR, "best.pth"))

        # Visual Check
        def to_np(t): return (t[0].squeeze().cpu().numpy() + 1) / 2
        fig, ax = plt.subplots(1, 3, figsize=(12, 4))
        ax[0].imshow(to_np(sar), cmap='gray'); ax[0].set_title("SAR")
        ax[1].imshow(to_np(target), cmap='gray'); ax[1].set_title("Target L")
        ax[2].imshow(to_np(fake_L), cmap='gray'); ax[2].set_title("Pred L")
        plt.savefig(os.path.join(CHECKPOINT_DIR, f"epoch_{epoch+1}.png"))
        plt.close()

In [3]:
if __name__ == "__main__":
    main()

--> No checkpoint found. Starting fresh.
Starting Structure Training on cuda...


Epoch 1/100 [LR=0.0002]: 100%|██████████| 900/900 [07:46<00:00,  1.93it/s, loss=0.354, lr=0.0002]


--> Val Metrics: PSNR: 14.64 | SSIM: 0.2171
    New Best SSIM! (0.2171). Saving best.pth...


Epoch 2/100 [LR=0.0002]: 100%|██████████| 900/900 [07:57<00:00,  1.89it/s, loss=0.274, lr=0.0002]


--> Val Metrics: PSNR: 14.96 | SSIM: 0.2248
    New Best SSIM! (0.2248). Saving best.pth...


...

--> Val Metrics: PSNR: 17.65 | SSIM: 0.3132


Epoch 99/100 [LR=1e-05]: 100%|██████████| 900/900 [09:35<00:00,  1.56it/s, loss=0.253, lr=1e-5]


--> Val Metrics: PSNR: 17.64 | SSIM: 0.3138
    New Best SSIM! (0.3138). Saving best.pth...


Epoch 100/100 [LR=1e-05]: 100%|██████████| 900/900 [09:27<00:00,  1.59it/s, loss=0.251, lr=1e-5]


--> Val Metrics: PSNR: 17.67 | SSIM: 0.3130
